In [1]:
"""
Baseline for CIL project on road segmentation.
This simple baseline consits of a CNN with two convolutional+pooling layers with a soft-max loss
"""

import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
from PIL import Image
from random import shuffle
import random

import code

import tensorflow.python.platform

import numpy as np
import tensorflow as tf

import cil_helper as cil

from skimage.feature import greycomatrix, greycoprops

In [2]:
try:
    tf.app.flags.DEFINE_string('log_dir', '/tmp/tensorflow_lukas',
                           """Directory where to write event logs """
                           """and checkpoint.""")
except: 
    print(tf.app.flags.FLAGS.log_dir)
    
FLAGS = tf.app.flags.FLAGS

In [3]:
TRAINING_SIZE = 40
TEST_SIZE = 20
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 64 # 64
NUM_EPOCHS = 20
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 20
ERROR_STEP = 50
TOTAL_STEP = TRAINING_SIZE * 2000

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!

NUM_CHANNELS = 6 # RGB, Saturation, Lightness, Edges
PATCH_SIZE = 16
EXTRA_CONTEXT = 32
CONTEXT_SIZE = EXTRA_CONTEXT + PATCH_SIZE + EXTRA_CONTEXT

# all prediction size
ALL_PER = 500

In [4]:
random.seed(SEED)

In [5]:
data_dir = 'training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 
test_dir = "test/"

In [6]:
def load_preproc(save_dir):
    assert save_dir.endswith('/')
    return np.load(save_dir + 'preprocessed2.npy')

def load_labels():
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, TRAINING_SIZE+1):
        imageid = "satImage_%.3d" % i
        image_filename = train_labels_filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    labels = np.asarray(gt_imgs, dtype=np.float32)
    return labels

def store_channel_to_img(chan):
    img = (chan * 255).astype('uint8')
    png = Image.fromarray(img)
    png.show
    return png

In [7]:
train_data = load_preproc(train_data_filename)
train_labels = load_labels()
print(train_data.shape)
print(train_labels.shape)

(100, 400, 400, 6)
(40, 400, 400)


In [8]:
# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step using the {feed_dict} argument to the Run() call below.
train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, CONTEXT_SIZE, CONTEXT_SIZE, NUM_CHANNELS),
    name='train_data')
train_labels_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, cil.NUM_LABELS),
    name='train_labels')

In [9]:
FILTER1_SIZE = 8
FILTER1_COUNT = 32
FILTER1_STRIDES = [1,1,1,1]

POOL1_FACTOR = 4

FILTER2_SIZE = 4
FILTER2_COUNT = 128
FILTER2_STRIDES = [1,1,1,1]

POOL2_FACTOR = 2

FC1_SIZE = 512

In [10]:
def model(data):
    filter1_shape = [FILTER1_SIZE, FILTER1_SIZE, NUM_CHANNELS, FILTER1_COUNT]
    filter1 = tf.Variable(tf.truncated_normal(filter1_shape, stddev=.1))
    conv1 = tf.nn.conv2d(data, filter1, FILTER1_STRIDES, 
                         padding='SAME', name='conv1')

    conv1_biases = tf.Variable(tf.constant(.1, shape=[FILTER1_COUNT]))

    relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

    pool1 = tf.nn.max_pool(relu1, ksize=[1,POOL1_FACTOR,POOL1_FACTOR,1], 
                           strides=[1,POOL1_FACTOR,POOL1_FACTOR,1], padding='SAME')
    ######################
    filter2_shape = [FILTER2_SIZE, FILTER2_SIZE, FILTER1_COUNT, FILTER2_COUNT]
    filter2 = tf.Variable(tf.truncated_normal(filter2_shape, stddev=.1))
    conv2 = tf.nn.conv2d(pool1, filter2, FILTER2_STRIDES, 
                         padding='SAME', name='conv2')

    conv2_biases = tf.Variable(tf.constant(.1, shape=[FILTER2_COUNT]))

    relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))

    pool2 = tf.nn.max_pool(relu2, ksize=[1,POOL2_FACTOR,POOL2_FACTOR,1], 
                           strides=[1,POOL2_FACTOR,POOL2_FACTOR,1], padding='SAME')
    ######################
    shp = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shp[0], shp[1] * shp[2] * shp[3]])

    fc1_weights = tf.Variable(
        tf.truncated_normal([reshape.get_shape().as_list()[1], FC1_SIZE], stddev=.1))
    fc1 = tf.matmul(reshape, fc1_weights)

    fc_last_weights = tf.Variable(
        tf.truncated_normal([FC1_SIZE, cil.NUM_LABELS], stddev=.1))

    fc_last = tf.matmul(fc1, fc_last_weights)

    return tf.nn.softmax(fc_last)

In [11]:
logits = model(train_data_node)
train_prediction = tf.nn.softmax(logits)

In [12]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, train_labels_node))

In [13]:
batch_idx = tf.Variable(0)
learning_rate = tf.train.exponential_decay(
    0.01,                # Base learning rate.
    batch_idx * BATCH_SIZE,  # Current index into the dataset.
    TRAINING_SIZE,          # Decay step.
    0.95,                # Decay rate.
    staircase=True)

In [21]:
#optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
    #loss, global_step=batch_idx)
optimizer = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss)

In [15]:
def get_window(image, y, x, truth=None):
    #padding the image with wrap-around
    window = np.take(np.take(image, 
                     range(y,y+CONTEXT_SIZE), axis=0,mode='wrap'), 
                     range(x,x+CONTEXT_SIZE), axis=1,mode='wrap')
    if not truth is None:
        truth_window = np.take(np.take(truth,
     range(y+EXTRA_CONTEXT,y+EXTRA_CONTEXT+PATCH_SIZE), axis=0,mode='wrap'),
     range(x+EXTRA_CONTEXT,x+EXTRA_CONTEXT+PATCH_SIZE), axis=1,mode='wrap')
        #list_patches.append(np.rot90(im_patch))
        #list_patches.append(np.fliplr(im_patch))
        return (window, truth_window)
    else:
        return window

def get_training_samples(batch_size):
    batch = np.empty((batch_size, CONTEXT_SIZE, CONTEXT_SIZE, NUM_CHANNELS))
    labels = np.empty((batch_size, cil.NUM_LABELS))
    for i in range(batch_size):
        pic_idx = random.randrange(0,TRAINING_SIZE)
        pic = train_data[pic_idx]
        y = random.randrange(0,pic.shape[0])
        x = random.randrange(0,pic.shape[1])
        
        window, truth = get_window(pic, y, x, train_labels[pic_idx])
        label = truth.mean()
        batch[i] = window
        labels[i][0] = label
        labels[i][1] = 1 - label
    return (batch, labels)

def get_patches(im):
    patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    for i in range(0-EXTRA_CONTEXT,imgheight,PATCH_SIZE):
        for j in range(0-EXTRA_CONTEXT,imgwidth,PATCH_SIZE):
            window = get_window(im, j, i)
            patches.append(window)
    return patches[34:36]

In [16]:
s = tf.InteractiveSession()

In [25]:
tf.initialize_all_variables().run()
print ('Initialized!')

Initialized!


In [26]:
# Restore variables from disk.
saver = tf.train.Saver()
#saver.restore(s,FLAGS.train_dir + "/model16-32.ckpt")

In [27]:
for epoch in range(NUM_EPOCHS):
    for i in range(TOTAL_STEP):
        batch, labels = get_training_samples(BATCH_SIZE)

        feed_dict = { train_data_node: batch, train_labels_node: labels}
        _, l, lr, predictions = s.run(
                        [optimizer, loss, learning_rate, train_prediction],
                        feed_dict=feed_dict) 
    
        stepNr = i + epoch*TOTAL_STEP
        if stepNr % ERROR_STEP == 0:
            format_str = ('step %d, error: %.1f%%')
            error = cil.error_rate(predictions, labels)
            print (format_str % (stepNr, error)) 
        elif stepNr % RECORDING_STEP == 0:
            format_str = ('step %d, batch error: %.1f%%')
            loss_value = cil.error_rate(predictions, labels)
            print (format_str % (stepNr, l)) 

step 0, error: 76.6%


KeyboardInterrupt: 

In [ ]:
#save_path = saver.save(s, FLAGS.train_dir + "/model16-32.ckpt")

In [28]:
# Get prediction overlaid on the original image for given input image
def get_prediction_with_overlay(img):
    
    img_prediction = get_prediction(img)
    oimg = cil.make_img_overlay(img[:,:,:3], img_prediction)

    return oimg

# Get a concatenation of the prediction and groundtruth for given input file
def get_prediction_with_groundtruth(img):
    
    img_prediction = get_prediction(img)
    cimg = cil.concatenate_images(img[:,:,:3], img_prediction)

    return cimg

# Get prediction for given input image 
def get_prediction(img):
    patches = get_patches(img)
    data = np.asarray(patches)
    print(data.shape)
    data_node = tf.constant(data)
    print(data_node.get_shape())
    output = tf.nn.softmax(model(data_node))
    print(output.get_shape())
    output_prediction = s.run(output)
    img_prediction = cil.label_to_img(img.shape[0], img.shape[1], IMG_PATCH_SIZE, IMG_PATCH_SIZE, output_prediction)

    return img_prediction

In [29]:
print ("Running prediction on training set")
prediction_training_dir = "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
    
for i in range(TRAINING_SIZE):
    pimg = get_prediction_with_groundtruth(train_data[i])
    Image.fromarray(pimg).save(prediction_training_dir + "prediction16_" + str(i+1) + ".png")
    oimg = get_prediction_with_overlay(train_preproc[i])
    oimg.save(prediction_training_dir + "overlay16-32_" + str(i+1) + ".png")  

Running prediction on training set
(2, 80, 80, 6)
(2, 80, 80, 6)
(2, 2)


FailedPreconditionError: Attempting to use uninitialized value Variable_12
	 [[Node: Variable_12/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_12"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_12)]]
Caused by op 'Variable_12/read', defined at:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/leo/.venvlis/lib/python3.4/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/leo/.venvlis/lib/python3.4/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/home/leo/.venvlis/lib/python3.4/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/home/leo/.venvlis/lib/python3.4/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/home/leo/

In [ ]:
print ("Running prediction on test set")
prediction_test_dir = "predictions_test/"
if not os.path.isdir(prediction_test_dir):
    os.mkdir(prediction_test_dir)

test_preproc = load_preproc(test_dir)
for i in range(TEST_SIZE):
    pred = get_prediction(test_preproc[i])
    pimg = cil.img_float_to_uint8(pred)
    Image.fromarray(pimg).save(prediction_test_dir + "prediction16_" + str(i+1) + ".png")
    oimg = cil.make_img_overlay(test_preproc[i], pred)
    oimg.save(prediction_test_dir + "overlay16-32_" + str(i+1) + ".png")

In [ ]:
import re
th = 0.25

In [ ]:
# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > th:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))


def save(submission_filename):
    #subm#ission_filename = 'submission_07.csv'
    image_filenames = []
    for i in range(1, 51):
        imagename = 'prediction_' + str(i)
        image_filename = 'predictions_test/' + imagename + '.png'
        print(image_filename)
        image_filenames.append(image_filename)
    masks_to_submission(submission_filename, *image_filenames)
    
save('submission_lukas_8-24.csv')